<center>
<img src="../../img/ods_stickers.jpg">

## Открытый курс по машинному обучению. Сессия № 3
***

## Индивидуальный проект по анализу данных на тему: 
# Прогнозирование оценки отеля путешественником на сайте TripAdvisor

#### Автор материала: Чигин Сергей Олегович (@chigin.sergey)

В данной работе предлагается исследовать датасет, содержащий информацию об отзывах с сайта TripAdvisor. 504 отзыва, сделанные в течение 2015 года, оценивают 21 гостиницу. Все гостиницы расположены на Лас-Вегас-Стрип (примерно семикилометровый участок бульвара Лас-Вегас в округе Кларк в штате Невада, США).

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

## 1. Описание набора данных и признаков

### 1.1 Процесс сбора даннах

Данные были взяты из репозитория UCI: https://archive.ics.uci.edu/ml/datasets/Las+Vegas+Strip

### 1.2 Описание признаков и целевой переменной

Признаки для простоты восприятия можно разбить на три группы: __характеристики пользователя__, __характеристики отзыва__ и __характеристики отеля__. Всего в датасете встречается три типа данных: __binary__ - бинарный, __numeric__ - количественный и __nominal__ - категориальный.  

__Характеристики пользователя__:
* __User country__ - гражданство пользователя, оставившего текущий отзыв (nominal: 48 стран);
* __User continent__ - континент, на котором находится страна пользователя (nominal: 6 континентов);
* __Member years__ - кол-во лет, в течение которых пользователь является зарегестрированным на TripAdvisor.com (numeric: от 0 до 13);
* __Nr. reviews__ - общее кол-во отзывов, сделанных пользователем на сайте (numeric: от 1 до 775);
* __Nr. hotel reviews__ - кол-во отзывов, посвященных отелям (numeric: от 0 до 263);
* __Helpful votes__ - общее кол-во лайков у пользователя (numeric: от 0 до 365).

__Характеристики отзыва__:
* __Review month__ - месяц, во время которого был оставлен отзыв (nominal: 12 месяцев);
* __Review weekday__ - день недели, в течение которого был оставлен отзыв (nominal: 7 дней недели);
* __Period of stay__ - промежуток времени, в который попало пребывание пользователя в отеле (nominal: 'Dec-Feb', 'Mar-May', 'Jun-Aug' и 'Sep-Nov');
* __Traveler type__ - тип путешествия (nominal: 'Couples', 'Families', 'Friends', 'Business' и 'Solo').

__Характеристики отеля__:
* __Hotel name__ - название отеля (nominal: 21 название отеля);
* __Hotel stars__ - кол-во звезд (numeric: от 3 до 5);
* __Nr. rooms__ - кол-во номеров в отеле (numeric: от 188 до 4027);
* __Pool__ - наличие бассейна (binary: yes/no);
* __Gym__ - наличие тренажерного зала (binary: yes/no);
* __Tennis court__ - наличие теннисного корта; (binary: yes/no)
* __Spa__ - наличие спа-салона (binary: yes/no);
* __Casino__ - наличие казино (binary: yes/no);
* __Free internet__ - наличие бесплатного интернета (binary: yes/no).

__Целевая переменная__:
* __Score__ - оценка отеля в текущем отзыве (numeric: от 1 до 5).

### 1.3 Описание решаемой задачи и ее ценность

Определение признаков, оказывающих наибольшее влияние на оценку отеля при составлении отзыва.

Средняя оценка на сайте, сформированная на основе отзывов, имеет большое значение для путешественника при выборе отеля.   Результаты текущего исследования могут быть полезными для менеджеров отелей в работе по улучшению среднего рейтинга, который оказывает влияние на заполняемость отеля. 

Изначально текущая задача является задачей __многоклассовой классификации__. Однако, в процессе исследования (см. п.2.2) задача сведется к __бинарной классификации__. 

## 2. Анализ данных

Выведем несколько первых строк и размер датасета.

In [ ]:
df=pd.read_csv('../../data/LasVegasTripAdvisorReviews-Dataset.csv',sep=';')
print('Shape:', df.shape)
df[0:3]

### 2.1 Типы признаков и проверка данных на пропуски

In [ ]:
df.info()

Видим, что в датасете нет пропусков - в каждом столбце по 504 наблюдения. __Hotel stars__ при формировании датафрейма определился как категориальный признак (object). Поищем возможную причину.  

In [ ]:
np.unique(df['Hotel stars'])

В числе возможных значений присутствуют вещественные числа, дробная часть которых записана через запятую. Именно это и является причиной. Сделаем небольшое преобразование и поменяем тип данных на численный.   

In [ ]:
df['Hotel stars']=df['Hotel stars'].apply(lambda x: x.replace(',','.')).astype('float64')

Все остальные признаки имеют указанный в описании (п.1.2) тип данных. 

In [ ]:
df.get_ftype_counts()

В итоге у нас имеется 7 количественных признаков (6 - целочисленных, 1 - вещественный) и 13 категориальных, среди которых 6 являются бинарными yes/no.

***
_В следующих подпунктах подробно исследована целевая переменная и признаковые группы. Анализ каждой группы последовательно проходит следующие этапы:_ 
* _вывод основных статистических показателей;_ 
* _визуализация распределений;_
* _оценка взаимодействий между признаками внутри группы;_
* _оценка влияния признаков на целевую переменную._

_Таким образом п.2 включает в себя одновременно и первичный анализ данных, и визуальный анализ данных, который по умолчанию должен был идти следующим пунктом. В связи с этим после п.2 сразу же идет п.4, где собраны и прокомментированны все найденные во время анализа зависимости._
***

### 2.2 Анализ целевой переменной

В большинстве случаев очень полезную информацию об отеле несет само содержание отзыва, из которого вытекает поставленная юзером оценка. К сожалению, в текущем датасете эта информация не представлена. Помимо этого у датасета очень небольшой объем - всего 504 объекта. В связи с этим было принято решение немного упростить задачу и свести ее к __бинарной классификации__: 1 - хороший отзыв (оценки 4 и 5), 0 - плохой отзыв (оценки 1, 2 и 3).

Посмотрим на распределение начальной целевой переменной __Score__, а затем на распределение новой __new_score__.

In [ ]:
sns.set(style="darkgrid", rc={'figure.figsize':(16,5)})
fig, ax =plt.subplots(1,2)
sns.countplot(df['Score'], ax=ax[0])
df['new_score']=df['Score'].apply(lambda x: 0 if x in [1,2,3] else 1)
sns.countplot(df['new_score'], ax=ax[1]);

In [ ]:
df['new_score'].value_counts(normalize=True)

Видно, что в распределении __new_score__ присутствует дисбаланс. Хороших отзывов примерно в 3.5 раза больше. Будем учитывать это при работе с моделью.

### 2.3 Анализ характеристик пользователя

#### Количественные признаки

Посмотрим на основные статистические параматеры.

In [ ]:
df[['User country', 'User continent','Nr. reviews', 'Nr. hotel reviews', 'Helpful votes', 'Member years']].describe()

Сразу же бросается в глаза отрицательное значение __min__ у __Member years__. Проверим, нет ли еще отрицательных значений у этого признака.  

In [ ]:
df.loc[df['Member years']<0, 'Member years']

Да, оно единственное. Скорее всего была просто допущена ошибка при формировании датасета. Исправим ее путем замены на медианное значение.

In [ ]:
df.loc[df['Member years']==df['Member years'].min(), 'Member years']=df['Member years'].median()

Сделаем pairplot для этих признаков. 

In [ ]:
sns.pairplot(df[['Nr. reviews', 'Nr. hotel reviews', 'Helpful votes', 'Member years']], size=2.7, diag_kind='hist');

У __Nr. reviews, Nr. hotel reviews__ и __Helpful votes__ распределения очень далеки от нормальных, у __Member years__ ситуация получше. По scatter plots видно, что __Member years__ с кем-то из признаков особо никак не взаимодействует. О взаимодействиях между остальными судить довольно сложно - точки слишком кучкуются около начала координат. Добавим к признакам целевую переменную __new_score__ и построим матрицу корреляций. Признаки численные - коэффициенты рассчитываем по Пирсону.  

In [ ]:
corr_matrix_user=df[['Nr. reviews', 'Nr. hotel reviews', 'Helpful votes', 'Member years', 'new_score']].corr(method='pearson')
sns.heatmap(corr_matrix_user, annot=True, fmt=".2f", linewidths=.5, cmap='YlGnBu');

Видим, что у __Nr. reviews, Nr. hotel reviews__ и __Helpful votes__ достаточно высокие коэффициенты. Это нормально. Логично предположить, что у юзера с большим кол-вом рецензий на сайте будет и большое кол-во отзывов непосредственно об отелях. Тоже самое и по лайкам. Их кол-во напрямую зависит от кол-ва рецензий. Целевая переменная ни с одним из признаков не коррелирует. 

#### Категориальные признаки

In [ ]:
df[['User country', 'User continent','Nr. reviews', 
    'Nr. hotel reviews', 'Helpful votes', 'Member years']].describe(include='object')

In [ ]:
sns.set(style="darkgrid", rc={'figure.figsize':(16,5)})
fig, ax =plt.subplots(1,2)
sns.countplot(df['User continent'], order=df['User continent'].value_counts().index, ax=ax[1])
top10=df[df['User country'].isin(df['User country'].value_counts().head(10).index)]['User country']
sns.countplot(top10, order=top10.value_counts().index, ax=ax[0]);

Для __User country__ выведены 10 самых популярных стран в датасете. Всего же представлено 48 стран. Вычислим какая доля отзывов приходится на топ-10 от общего числа.

In [ ]:
print('Share of top 10 countries:', len(top10)/df.shape[0])

Создадим __User country new__, немного преобразовав __User country__ путем сокращения кол-ва уникальных значений до 11. В них будут входить определенные выше старна топ-10 и новое значение other, объединяющее оставшиеся 38 стран, на чью долю приходится около 13% отзывов от общего числа.

In [ ]:
df['User country new']=df['User country'].apply(lambda x: 'Other' if x not in np.unique(top10) else x)

Изобразим распределение хороших и плохих отзывов и по __User continent__, и по __User country new__.

In [ ]:
sns.set(style="darkgrid", rc={'figure.figsize':(16,10)})
fig, ax =plt.subplots(2,1)
sns.countplot(x='User continent', hue='new_score', data=df, order=df['User continent'].value_counts().index, ax=ax[0]);
sns.countplot(x='User country new', hue='new_score', data=df, 
              order=df['User country new'].value_counts().index, ax=ax[1]);

Здесь стоит вспомнить зафиксированную в п.2.2 долю отрицательных отзывов в 22,5% от общего кол-ва во всем датасете. Из распределений выше видно, что в абсолютном бол-ве объектов доля негативных отзывов примерно такая же или ниже. Из континентов исключениями являются Азия и Африка, из стран - Канада, Германия, Египет и группа стран, не вошедших в топ-10. У них доля плохих отзывов выше.  

In [ ]:
print('Доли плохих отзывов среди континентов в порядке убывания:')
1-df.groupby([df['User continent']])['new_score'].agg([np.mean]).sort_values(by='mean', ascending=True).T

In [ ]:
print('Доли плохих отзывов среди стран в порядке убывания:')
1-df.groupby([df['User country new']])['new_score'].agg([np.mean]).sort_values(by='mean', ascending=True).T

Но на страны или континенты, которые являются исключениями, приходится довольно небольшое кол-во отзывов всего. Тот факт, что у них высокие доли отрицатальных отзывов нельзя считать четкой зависимостью.    

### 2.4 Анализ характеристик отзыва

Все признаки, характеризующие отзыв, имеют __категориальный__ тип данных. Посмотрим на их общую статистику.    

In [ ]:
df[['Period of stay', 'Traveler type', 'Review month', 'Review weekday']].describe(include='object')

__Period of stay__ и __Review month__ определяют временной отрезок, за который у нас имеются данные. Выведем распределение отзывов по ним.

In [ ]:
df['Period of stay'].value_counts()

In [ ]:
df['Review month'].value_counts()

За каждый месяц в 2015 году было взято ровно по 42 отзыва. По сезонам отзывы распределены примерно равномерно. 

Визуализируем распределения оставшихся двух признаков (__Traveler type__ и __Review weekday__).

In [ ]:
sns.set(style="darkgrid", rc={'figure.figsize':(16,5)})
fig, ax =plt.subplots(1,2)
sns.countplot(df['Traveler type'], ax=ax[0])
sns.countplot(df['Review weekday'], ax=ax[1]);

Посмотрим на то, как меняется кол-во отдыхающих каждого типа от сезона к сезону.

In [ ]:
sns.countplot(x='Period of stay', hue='Traveler type', data=df);

Семьи чаще выбираются на отдых летом, в это же время падает кол-во бизнес-трипов. В зимнее время друзья путешествуют вместе реже, чем в остальные сезоны.

Построим похожий countplot для __Review month__ и __Period of stay__. Интересно посмотреть есть ли значительные отставания между написанием отзыва и временем пребывания в отеле.  

In [ ]:
sns.set(style="darkgrid", rc={'figure.figsize':(16,5)})
sns.countplot(x='Review month', hue='Period of stay', data=df);

Очень странная ситуация с февралем. Есть несколько отзывов, которые были написаны в этом месяце, хотя период пребывания в отеле значится __Mar-May__. Как будто отзыв об отеле был составлен еще до самого пребывания там. Выведем отдельно эти отзывы. С остальными месяцами все в порядке.  

In [ ]:
df[(df['Review month']=='February') & (df['Period of stay']=='Mar-May')]

Оба отзыва являются положительными. Скорее всего была просто допущена ошибка при формировании датасета. Чуть выше мы выводили распределение отзывов по сезонам, оно было практически равномерным. Сделаем это еще раз.

In [ ]:
df['Period of stay'].value_counts()

Очевидно, что распределение было бы полностью равномерным, если бы не эта ошибка с двумя отзывами. Исправим ее, поменяв у них __Period of stay__ на __Dec-Feb__.

In [ ]:
df.loc[(df['Review month']=='February') & (df['Period of stay']=='Mar-May'), 'Period of stay']='Dec-Feb'

Построим countplot с распределениями __new_score__ для __Period of stay__ и __Review month__. 

In [ ]:
sns.set(style="darkgrid", rc={'figure.figsize':(16,8)})
fig, ax =plt.subplots(2,1)
sns.countplot(x='Review month', hue='new_score', data=df, ax=ax[0]);
sns.countplot(x='Period of stay', hue='new_score', data=df, ax=ax[1]);

Смотря на диаграмму по месяцам, стоит вспомнить, что кол-во отзывов в течение каждого месяца одинаково и равно 42. На май приходится самое большое кол-во плохих отзывов. 

Диаграмма по __Period of stay__ по сути является более грубой версией диаграммы по __Review month__. После исправления ошибки в данных и проверки на отсутствие значительных лагов между пребыванием в отеле и написанием отзыва стало понятно, что эти два признака дублируют друг друга. Разница только во временном отрезке (месяц у __Review month__ и три месяца у __Period of stay__). При предобратке данных признак __Period of stay__ будет необходимо удалить.

Посмотрим на доли плохих отзывов по __Traveler type__ и __Review weekday__.

In [ ]:
print('Доли плохих отзывов среди типов путешествий в порядке убывания:')
1-df.groupby([df['Traveler type']])['new_score'].agg([np.mean]).sort_values(by='mean', ascending=True).T

Выше всего у __Families__, но на первом месте они с небольшим отрывом. От __Traveler type__ сильно распределение целевой переменной не зависит.

In [ ]:
print('Доли плохих отзывов среди дней недели написания отзыва в порядке убывания:')
1-df.groupby([df['Review weekday']])['new_score'].agg([np.mean]).sort_values(by='mean', ascending=True).T

Примерно тоже самое можно сказать и про __Review weekday__. Нет какого-то конкретного дня, который аккумулировал на себе прям сильно большую долю плохих отзывов в сравнении с другими днями.

### 2.5 Анализ характеристик отеля

#### Категориальные признаки

In [ ]:
df[['Hotel name', 'Pool', 'Gym', 'Tennis court', 'Spa', 'Casino', 'Free internet', 
    'Hotel stars', 'Nr. rooms']].describe(include='object')

Цифры по __Hotel name__ подтверждают анонсированное в описании распределение - суммарно 504 отзыва о 21 отеле. Посмотрим каким образом распределились отзывы среди этого признака. 

In [ ]:
df['Hotel name'].value_counts()

Они распределены равномерно - по 24 на каждый.

Визуализируем распределения 6-ти бинарных (yes/no) признаков, отвечающих за гостиничные услуги.

In [ ]:
sns.set(style="darkgrid", rc={'figure.figsize':(16,8)})
fig, ax =plt.subplots(nrows=2, ncols=3, sharex='all', sharey='all')
sns.countplot(df['Pool'], ax=ax[0,0])
sns.countplot(df['Gym'], ax=ax[0,1])
sns.countplot(df['Tennis court'], ax=ax[0,2])
sns.countplot(df['Spa'], ax=ax[1,0])
sns.countplot(df['Casino'], ax=ax[1,1])
sns.countplot(df['Free internet'], ax=ax[1,2]);

Бассейн и спа в большинстве отелей отсутствует. Стоит также отметить, что есть отели, пускай их и не много, у которых нет бесплатного интернета.

#### Количественные признаки

In [ ]:
df[['Hotel name', 'Pool', 'Gym', 'Tennis court', 'Spa', 'Casino', 'Free internet', 
    'Hotel stars', 'Nr. rooms']].describe()

Сразу посмотрим на распределение по __Hotel stars__ и выведем кол-во уникальных значений у __Nr. rooms__.

In [ ]:
sns.set(style="darkgrid", rc={'figure.figsize':(8,5)})
sns.countplot(df['Hotel stars']);

In [ ]:
len(np.unique(df['Nr. rooms']))

21 уникальное значение говорит о том, что все гостиницы имеют разные кол-ва номеров. Посмотрим на их распределение.

In [ ]:
sns.distplot(df['Nr. rooms']);

#### Взаимодействия признаков и влияние на целевую переменную

Перекодируем бинарные признаки yes/no и построим матрицу корреляций.

In [ ]:
binary_features=['Pool', 'Gym', 'Tennis court', 'Spa','Casino', 'Free internet']
data_bin=df[binary_features]
data_bin_dum=pd.get_dummies(data_bin)
data_bin_cut=data_bin_dum.iloc[:,1::2]
df_3=pd.concat([data_bin_cut, df[['Hotel stars', 'Nr. rooms', 'new_score']]],axis=1)

Бинарные признаки были разбиты на взаимодополняющие пары (пример: __Pool_YES__ и __Pool_NO__). Из каждой пары были удалены по одному признаку, посколько они не добавляют никакой информации. 

In [ ]:
corr_matrix_hotel=df_3.corr()
sns.heatmap(corr_matrix_hotel, annot=True, fmt=".2f", linewidths=.5, cmap='YlGnBu');

__new_score__, пускай коэффициенты и невысокие, но коррелирует с тремя признаками: __Hotel stars__, __Free internet_YES__ и __Pool_YES__.

Так же в матрице наблюдается довольно большое кол-во относительно высоких коэффициентов между признаками. Логика в этом простая. Условно, если есть в отеле казино, то наверняка в нем будет и спа, и спортзал и тд. Но среди признаков нет тех, которые бы дублировали друг друга. 

## 4. Инсайты, найденные зависимости

* Целевая переменая __new_score__ не сбалансирована. Хороших отзывов примерно в 3.5 раза больше, чем плохих;
* Признак __Helpful votes__ сильно коррелирует с __Nr. reviews__ и __Nr. hotel reviews__ (0.79 и 0.76 соответственно) - его можно отбросить;
* __User country__/__User country new__ и __User continent__ несут в себе одинаковую информацию. Один признак распределяет объекты на 48/11 категорий, второй - на 6. При этом оба признака не имеют большого влияния на __new_score__. Первый признак уберем, а второй (__User continent__) оставим. При его кодировании признаковое пространство не будет сильно раздуто;
* Аналогичная ситуация с __Review month__ и __Period of stay__. Оставляем только __Review month__;
* Была обнаружена некая корреляция с __Hotel stars__, __Free internet__ и __Pool__. Логично предположить, что класс отеля и наличие услуг, которые обеспечивают комфорт и удобство клиентов, напрямую влияют на впечатление от пребывания в нем;
* Признак __Hotel name__ содержит только названия отелей и он не несет никакой дополнительной информации о самом отеле, его можно убрать.


## 5. Выбор метрики

Мы решаем задачу бинарной классификации. Допустимые метрики качества в таком случае:
* Accuracy
* Precision
* Recall
* F1-score
* ROC-AUC
* Logistic loss

В нашей задаче отсутствует баланс классов. В этом случае наилучшей метрикой для оценки качества модели будет __ROC-AUC__. Она позволит оценить модель вцелом, не привязываясь к какому-то конкретному порогу.

## 6. Выбор модели

Мои текущие знания моделей на текущий момент не очень обширные. Я в Data Science - новичок. Знаком только с теми, которые были рассмотрены в рамках курса. Среди них следующие алгоритмы подходят для задачи бинарной классификации: 
* KNeighborsClassifier()
* DecisionTreeClassifier()
* RandomForestClassifier()   
* LogisticRegression()
* XGBCClassifier()

Попробуем их с базовыми настройками. Потом отберем те, которые показали лучшие результаты, и продолжим исследование уже только с ними.

## 7. Предобработка данных

Во время анализа данных уже были сделаны небольшие преобраование и исправлены несколько ошибок. В п.4 были перечислены признаки, которые использоваться не будут. Соберем в отдельный датафрейм те, с которыми будем работать, и разобьем их на две группы: количественные и категориальные.

In [ ]:
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split, GridSearchCV 

final_df=df[['Nr. reviews', 'Nr. hotel reviews', 'Member years', 'Hotel stars', 'Nr. rooms',
            'User continent', 'Review month', 'Review weekday', 'Traveler type',
                         'Pool', 'Gym', 'Tennis court', 'Spa','Casino', 'Free internet','new_score']]

cat_features=['User continent', 'Review month', 'Review weekday', 'Traveler type',
                         'Pool', 'Gym', 'Tennis court', 'Spa','Casino', 'Free internet']

num_features=['Nr. reviews', 'Nr. hotel reviews', 'Member years', 'Hotel stars', 'Nr. rooms']

Разобьем данные на тренировочную, валидационную и тестовую выборки. Разбивку будем делать с учетом текущего баланса классов.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(final_df.drop('new_score', axis=1), final_df['new_score'], 
                                                    test_size=0.2, random_state=17, stratify=final_df['new_score'])
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=17)

In [ ]:
X_train.shape, X_valid.shape, X_test.shape

Количественные признаки отмасшатабируем с помощью __StandardScaler()__. Категориальные признаки преобразуем с помощью __dummy__ кодирования.

In [ ]:
data_scale_train = X_train[num_features]
data_scale_test = X_valid[num_features]
scaler = StandardScaler()
data_scaled_train = scaler.fit_transform(data_scale_train)
data_scaled_valid = scaler.transform(data_scale_test)

In [ ]:
data_dum_train = X_train[cat_features]
data_dum_valid = X_valid[cat_features]

data_dum_full=pd.concat([data_dum_train, data_dum_valid], axis=0)
idx_split = data_dum_train.shape[0]

data_dummy_full=pd.get_dummies(data_dum_full)
data_dummy_train = data_dummy_full[:idx_split]
data_dummy_valid = data_dummy_full[idx_split:]

Объедим уже преобразованные признаки и приведем их к виду разряженных матриц.

In [ ]:
from scipy.sparse import csr_matrix
X_train_sparse = csr_matrix(hstack([data_scaled_train, data_dummy_train]))

X_valid_sparse = csr_matrix(hstack([data_scaled_valid, data_dummy_valid])) 

## 8. Кросс-валидация и настройка гиперпараметров модели

### 8-1. Кросс-валидация

На кросс-валидации будем разбивать тренировочную выборку на 5 фолдов с учетом баланса классов.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier


from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

models = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(random_state = 17),
    RandomForestClassifier(n_estimators=100, random_state = 17), 
    XGBClassifier(random_state = 17),
    LogisticRegression(random_state = 17)
    
]

for model in models:
    model.fit(X_train_sparse, y_train)
    score = np.mean(cross_val_score(model, X_train_sparse, y_train, cv=skf, scoring='roc_auc'))
    print('roc auc:', score)
    print(model)
    print()

Вцелом скор довольно невысокий у всех, но лучший у __LogisticRegression__. Оставляем ее и попробуем улучшить как-то качество с помощью работы над признаковым пространством.

## 9. Создание новых признаков и описание этого процесса

В данных преимущественно категориальные признаки. Мощной техникой для работы в этом случае является учет попарных взаимодействий признаков (feature interactions). Построим попарные взаимодействия всех категориальных признаков.

### 9.1 Попарные взаимодействия признаков

In [ ]:
train_interact = X_train[['User continent', 'Review month', 'Review weekday', 'Traveler type','Pool', 
                       'Gym', 'Tennis court', 'Spa','Casino', 'Free internet']]

cat_features = train_interact.columns[train_interact.dtypes == 'object']
for i, col1 in enumerate(cat_features):
    for j, col2 in enumerate(cat_features[i + 1:]):
        train_interact[col1 + '_' + col2] = train_interact[col1] + '_' + train_interact[col2] 

In [ ]:
train_interact.head()

In [ ]:
dummy_train_interact = pd.get_dummies(train_interact)
X_train_interact_sparse = csr_matrix(hstack([data_scaled_train, dummy_train_interact]))
X_train_interact_sparse.shape

Ох, после объединения с 5-ю отскалированными количественными получилось 735 признаков - это больше чем тренировочных объектов. Вряд ли это приведет к каким-то улучшениями, но все равно попробуем.

In [ ]:
model=LogisticRegression(random_state = 17)

model.fit(X_train_sparse, y_train)
score = np.mean(cross_val_score(model, X_train_interact_sparse, y_train, cv=skf, scoring='roc_auc'))
print('roc auc:', score)
print(model)
print()

Да, скор упал. Но у нас есть еще 5 количественных признаков, двое из которых имели распределения с сильным скосом вправо.  

### 9.2 Логарифмирование количественных признаков

Посмотрим как изменятся распределения __Nr. reviews__ и __Nr. hotel reviews__ после логарифмирования.

In [ ]:
np.log(final_df[['Nr. reviews', 'Nr. hotel reviews']]+1).hist();

Стали очень похожи на нормальные распределения. Попробуем посмотреть на скор с прологарифмированными __Nr. reviews__ и __Nr. hotel reviews__.

In [ ]:
X_train[['Nr. reviews', 'Nr. hotel reviews']]=np.log(X_train[['Nr. reviews', 'Nr. hotel reviews']]+1)
X_valid[['Nr. reviews', 'Nr. hotel reviews']]=np.log(X_valid[['Nr. reviews', 'Nr. hotel reviews']]+1)

data_scale_train = X_train[num_features]
data_scale_valid = X_valid[num_features]
scaler = StandardScaler()
data_scaled_train = scaler.fit_transform(data_scale_train)
data_scaled_valid = scaler.transform(data_scale_valid)

data_dum_train = X_train[cat_features]
data_dum_valid = X_valid[cat_features]

data_dum_full=pd.concat([data_dum_train, data_dum_valid], axis=0)
idx_split = data_dum_train.shape[0]

data_dummy_full=pd.get_dummies(data_dum_full)
data_dummy_train = data_dummy_full[:idx_split]
data_dummy_valid = data_dummy_full[idx_split:]

In [ ]:
X_train_sparse = csr_matrix(hstack([data_scaled_train, data_dummy_train]))

X_valid_sparse = csr_matrix(hstack([data_scaled_valid, data_dummy_valid])) 

In [ ]:
model.fit(X_train_sparse, y_train)
score = np.mean(cross_val_score(model, X_train_sparse, y_train, cv=skf, scoring='roc_auc'))
print('roc auc:', score)
print(model)
print()

Есть небольшие улучшения, скор подрос. Попробуем понастраивать гиперпараметры.

## 8. Кросс-валидация и настройка гиперпараметров модели

### 8-2. Настройка гиперпараметров модели

У LogisticRegression() будем настраивать следующий параметр:
* С - обратное значение коэффициента регуляризации.

In [ ]:
params={'C': np.logspace(-3, 1, 10)}

model = LogisticRegression(random_state=17)

lr_grid = GridSearchCV(model, param_grid=params, cv=skf, scoring='roc_auc', verbose=True, n_jobs=-1)
lr_grid.fit(X_train_sparse,y_train)
print(lr_grid.best_score_)
print(lr_grid.best_params_)

Проверим лучшую модель на валидационной выборке.

In [ ]:
logit=lr_grid.best_estimator_

logit.fit(X_train_sparse, y_train)
logit_valid_pred = logit.predict_proba(X_valid_sparse)[:, 1]
roc_auc_score(y_valid, logit_valid_pred)

In [ ]:
logit

## 10. Построение кривых валидации и обучения

Сначала посмотрим на кривые обучения.

In [ ]:
from sklearn.model_selection import learning_curve

def plot_with_err(x, data, **kwargs):
    mu, std = data.mean(1), data.std(1)
    lines = plt.plot(x, mu, '-', **kwargs)
    plt.fill_between(x, mu - std, mu + std, edgecolor='none',
                     facecolor=lines[0].get_color(), alpha=0.2)

def plot_learning_curve():
    train_sizes = np.linspace(0.05, 1, 20)
    
    N_train, val_train, val_test = learning_curve(logit, X_train_sparse, y_train, train_sizes=train_sizes, cv=skf,
                                                  scoring='roc_auc')
    plot_with_err(N_train, val_train, label='training scores')
    plot_with_err(N_train, val_test, label='validation scores')
    plt.xlabel('Training examples'); 
    plt.ylabel('ROC-AUC')
    plt.legend()

In [ ]:
plt.figure(figsize=(14, 8))
plot_learning_curve()

В принципе, наблюдается хорошая тенденция – кривые постепенно сходятся. Можно предположить, что при дальнейшем движении направо качество на валидации можно будет повысить. Переходим к кривым валидации. 

In [ ]:
from sklearn.model_selection import validation_curve

C_range=np.logspace(-3, 1, 10)

val_train, val_test = validation_curve(model, X_train_sparse, y_train, param_name='C', 
                                       param_range=C_range, cv=skf, scoring='roc_auc')

plt.figure(figsize=(14, 8))
plot_with_err(C_range, val_train, label='training scores')
plot_with_err(C_range, val_test, label='validation scores')
plt.xlabel('C')
plt.ylabel('ROC AUC')
plt.legend();

У С есть явный пик. При увеличении параметра скор падает, но не резко и не значительно.

## 11. Прогноз на тестовой выборке

Объедими тренировочную выборку с валидационной и обучим на них модель. Затем проверим ее качество на тестовой выборке.

In [ ]:
X_train = X_train.append(X_valid)
y_train = y_train.append(y_valid)

In [ ]:
X_test[['Nr. reviews', 'Nr. hotel reviews']]=np.log(X_test[['Nr. reviews', 'Nr. hotel reviews']]+1)

data_scale_train = X_train[num_features]
data_scale_test = X_test[num_features]
scaler = StandardScaler()
data_scaled_train = scaler.fit_transform(data_scale_train)
data_scaled_test = scaler.transform(data_scale_test)

data_dum_train = X_train[cat_features]
data_dum_test = X_test[cat_features]

data_dum_full=pd.concat([data_dum_train, data_dum_test], axis=0)
idx_split = data_dum_train.shape[0]

data_dummy_full=pd.get_dummies(data_dum_full)
data_dummy_train = data_dummy_full[:idx_split]
data_dummy_test = data_dummy_full[idx_split:]

In [ ]:
X_train_sparse = csr_matrix(hstack([data_scaled_train, data_dummy_train]))

X_test_sparse = csr_matrix(hstack([data_scaled_test, data_dummy_test])) 

In [ ]:
logit.fit(X_train_sparse, y_train)
logit_test_pred = logit.predict_proba(X_test_sparse)[:, 1]
roc_auc_score(y_test, logit_test_pred)

Результат на тестовой выборке лучше. Думаю, что причина кроется в большем кол-ве данных, на которых модель обучалась. Мы словно проверили на практике предположение, сделанное по кривым обучения в п.10. Как раз объединив тренировочную выборку с валидационной, мы сместились немного вправо. 

## 12. Выводы

Изначально целью иследования было определение факторов, которые сильнее всего оказывают влияние на оценку. Давайте посмотрим на веса модели.

In [ ]:
names=data_scale_train.columns.append(data_dummy_full.columns)

def getCoefficients(model):
    """
    Возвращает отсортированные по абсолютному значению коэффициенты модели
    """
    
    coefs = pd.DataFrame(model.coef_[0], names)
    coefs.columns = ["coef"]
    coefs["abs"] = coefs.coef.apply(np.abs)
    return coefs.sort_values(by="abs", ascending=False).drop(["abs"], axis=1)    
    

def plotCoefficients(model):
    """
    Отрисовывает отсортированные по абсолютному значению коэффициенты модели
    """
    coefs = getCoefficients(model)
    
    plt.figure(figsize=(20, 10))
    coefs.coef.plot(kind='bar')
    plt.grid(True, axis='y')
    plt.hlines(y=0, xmin=0, xmax=len(coefs), linestyles='dashed');

In [ ]:
plotCoefficients(logit)

Результаты вряд ли можно назвать удивительными. Сильнее всего влияет на оценку кол-во звезд у отеля и наличи бесплатного интернета. 

На данный момент, качество модели не очень высокое, но убежден, что у исследования есть потенциал. Следующие пункты, наверняка помогли бы его улучшить: 
* увеличение объема данных (здесь речь непосредственно о кол-ве объектов в датасете);
* добавление новых признаков с полезной информацией (текст отзыва, год постройки отеля, численность обслуживающего персонала, языки, на которых говорит персонал и тд);
* охват больших территорий, на которых расположены отели (разные города, разные страны).